# Zarr Cloud native format workflow

Invoke a Common Workflow Language Workflow to generate the STAC Zarr

This notebook is linked to: https://eoap.github.io/zarr-cloud-native-format/cwl-workflow/zarr



## Setup

In [17]:
export WORKSPACE=/workspace/zarr-cloud-native-format
export RUNTIME=${WORKSPACE}/runs
mkdir -p ${RUNTIME}
cd ${RUNTIME}
oras pull ghcr.io/eoap/zarr-cloud-native-format/app-water-bodies:latest --output ${WORKSPACE}

ls -l ${WORKSPACE}/app-water-bodies.cwl

78
78
78
78
78✓ Pulled      app-water-bodies.cwl                   12.2/12.2 kB 100.00%    2ms
8  └─ sha256:bee4cb87f4a5549003c10f4be0be4991f46f687cb0653ed562a089ee5f62014d
8✓ Pulled      application/vnd.oci.image.manifest.v1+j. 559/559  B 100.00%     0s
8  └─ sha256:6f257baaef69e6e5221425c91addcc5d2fb459b9e5b7f3f0075556de3c2a5605
8Pulled [registry] ghcr.io/eoap/zarr-cloud-native-format/app-water-bodies:latest
Digest: sha256:6f257baaef69e6e5221425c91addcc5d2fb459b9e5b7f3f0075556de3c2a5605
-rw-rw-r-- 1 fbrito fbrito 12541 Jan 20 10:33 /workspace/zarr-cloud-native-format/app-water-bodies.cwl


## Run the Zarr Cloud native format workflow

Inspect and use `cwltool` to run the Zarr Cloud native format workflow definition:


In [18]:
cat ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl | yq e '.["$graph"][0]' -

class: Workflow
id: water-bodies
label: Water bodies detection based on NDWI and otsu threshold
doc: Water bodies detection based on NDWI and otsu threshold applied to Sentinel-2 COG STAC items
requirements:
  - class: ScatterFeatureRequirement
  - class: SubworkflowFeatureRequirement
  - class: SchemaDefRequirement
    types:
      - $import: https://raw.githubusercontent.com/eoap/schemas/main/string_format.yaml
      - $import: https://raw.githubusercontent.com/eoap/schemas/main/geojson.yaml
      - $import: |-
          https://raw.githubusercontent.com/eoap/schemas/main/experimental/api-endpoint.yaml
      - $import: https://raw.githubusercontent.com/eoap/schemas/main/experimental/discovery.yaml
inputs:
  stac_api_endpoint:
    label: STAC API endpoint
    doc: STAC API endpoint
    type: |-
      https://raw.githubusercontent.com/eoap/schemas/main/experimental/api-endpoint.yaml#APIEndpoint
  search_request:
    label: STAC search request
    doc: STAC search request
    type: |-
 

Run the CWL description, but first prepare the parameters.

Let's build the job parameters file

In [19]:
cat <<'EOF' > zarr-cloud-native-params.yaml
bands:
- green
- nir
search_request:
  bbox:
  - -121.399
  - 39.834
  - -120.74
  - 40.472
  collections:
  - sentinel-2-l2a
  datetime_interval:
    start:
      value: '2021-06-01T00:00:00'
    end:
      value: '2021-07-15T23:59:59'
  limit: 20
  max-items: 2
stac_api_endpoint:
  headers: []
  url:
    value: https://earth-search.aws.element84.com/v1/
EOF

cat zarr-cloud-native-params.yaml | yq .

bands:
  - green
  - nir
search_request:
  bbox:
    - -121.399
    - 39.834
    - -120.74
    - 40.472
  collections:
    - sentinel-2-l2a
  datetime_interval:
    start:
      value: '2021-06-01T00:00:00'
    end:
      value: '2021-07-15T23:59:59'
  limit: 20
  max-items: 2
stac_api_endpoint:
  headers: []
  url:
    value: https://earth-search.aws.element84.com/v1/


In [20]:
cwltool \
    --podman \
    --parallel \
    --outdir ${WORKSPACE}/runs \
    ${WORKSPACE}/cwl-workflow/app-water-bodies.cwl#water-bodies \
    zarr-cloud-native-params.yaml > zarr-cloud-native-results.json 2> zarr-cloud-native.log

Let's look at the content of the stderr:

In [21]:
cat zarr-cloud-native.log | egrep -v "WARNING|JSHINT"

INFO /home/fbrito/.local/bin/cwltool 3.1.20250110105449
INFO Resolved '/workspace/zarr-cloud-native-format/cwl-workflow/app-water-bodies.cwl#water-bodies' to 'file:///workspace/zarr-cloud-native-format/cwl-workflow/app-water-bodies.cwl#water-bodies'
INFO [workflow ] starting step discovery
INFO [workflow ] start
INFO [step discovery] start
INFO [job discovery] /tmp/q23b3xiv$ podman \
    run \
    -i \
    --userns=keep-id \
    --mount=type=bind,source=/tmp/q23b3xiv,target=/kurFwI \
    --mount=type=bind,source=/tmp/_fwp5hnv,target=/tmp \
    --workdir=/kurFwI \
    --read-only=true \
    --user=1000:1000 \
    --rm \
    --cidfile=/tmp/0mfyf84v/20260120103357-475374.cid \
    --env=TMPDIR=/tmp \
    --env=HOME=/kurFwI \
    ghcr.io/eoap/schemas/stac-api-client@sha256:a7e346f704836d07f5dabc6b29ee3359e7253f4a294d74f3899973b8920da6f7 \
    stac-client \
    search \
    https://earth-search.aws.element84.com/v1/ \
    --collections \
    sentinel-2-l2a \
    --bbox \
    -121.399 \
    

Let's inspect the stdout produced. 

In [22]:
cat zarr-cloud-native-results.json | jq  '.zarr_stac_catalog.path' -

"/workspace/zarr-cloud-native-format/runs/5pmu70kn"


In [23]:
tree $( cat zarr-cloud-native-results.json | jq -r '.zarr_stac_catalog.path' - ) 

/workspace/zarr-cloud-native-format/runs/5pmu70kn
├── catalog.json
└── water-bodies
    ├── collection.json
    └── water-bodies.zarr
        ├── measurements
        │   ├── ndwi
        │   │   ├── c
        │   │   │   └── 0
        │   │   │       ├── 0
        │   │   │       │   ├── 0
        │   │   │       │   ├── 1
        │   │   │       │   ├── 10
        │   │   │       │   ├── 2
        │   │   │       │   ├── 3
        │   │   │       │   ├── 4
        │   │   │       │   ├── 5
        │   │   │       │   ├── 6
        │   │   │       │   ├── 7
        │   │   │       │   ├── 8
        │   │   │       │   └── 9
        │   │   │       ├── 1
        │   │   │       │   ├── 0
        │   │   │       │   ├── 1
        │   │   │       │   ├── 10
        │   │   │       │   ├── 2
        │   │   │       │   ├── 3
        │   │   │       │   ├── 4
        │   │   │       │   ├── 5
        │   │   │       │   ├── 6
        │   │   │       │   ├── 7
        │   │   │       │   ├─

In [24]:
stac describe $( cat zarr-cloud-native-results.json | jq -r '.zarr_stac_catalog.path' - )/catalog.json

* <Catalog id=water-bodies>
    * <Collection id=water-bodies>


In [9]:
jq . $( cat zarr-cloud-native-results.json | jq -r '.zarr_stac_catalog.path' - )/water-bodies/collection.json

{
  "type": "Collection",
  "id": "water-bodies",
  "stac_version": "1.1.0",
  "description": "Detected water bodies",
  "links": [
    {
      "rel": "root",
      "href": "../catalog.json",
      "type": "application/json",
      "title": "Water bodies"
    },
    {
      "rel": "store",
      "href": "water-bodies.zarr",
      "type": "application/vnd.zarr; version=3",
      "title": "Zarr store for Water bodies"
    },
    {
      "rel": "parent",
      "href": "../catalog.json",
      "type": "application/json",
      "title": "Water bodies"
    }
  ],
  "stac_extensions": [
    "https://stac-extensions.github.io/projection/v2.0.0/schema.json",
    "https://stac-extensions.github.io/raster/v1.1.0/schema.json",
    "https://stac-extensions.github.io/datacube/v2.2.0/schema.json"
  ],
  "title": "Water bodies",
  "extent": {
    "spatial": {
      "bbox": [
        [
          -121.39905410179915,
          39.82336095080462,
          -120.73995321724426,
          40.48279883772837